In [13]:
calculate_emissions_json = {
    "Kalkamonsalpeter 27": {
        "Zusammensetzung": 0.27,
        "Emissionsfaktor": 7.101
    },
    "Harnstoff": {
        "Zusammensetzung": 0.46,
        "Emissionsfaktor": 12.4
    },
    "ASS 26N": {
        "Zusammensetzung": 0.26,
        "Emissionsfaktor": 11.438
    },
    "DAP": {
        "Zusammensetzung": {
            "N": 0.18,
            "P2O5": 0.46
        },
        "Emissionsfaktor": 11.958
    },
    "Gülle (Organisch)": {
        "Zusammensetzung": 1.0,
        "Emissionsfaktor": 4.97
    },
    "Menge" : {
        "Zusammensetzung": 1.0,
        "Emissionsfaktor": 4.97
    },
    "Curbatur + Caramba + Karate" : {
        "Zusammensetzung" : 0.007875,
        "Emissionsfaktor" : 8.674
    },
    "Mertil + Herbosol" : {
        "Zusammensetzung" : 0.005,
        "Emissionsfaktor" : 8.735
    },
    "Broadway Plus": {
        "Zusammensetzung" : 0.0048,
        "Emissionsfaktor" : 8.735 
    },
    "Elatus Era + Countdown+U46M" : {
        "Zusammensetzung" : 0.0153,
        "Emissionsfaktor" : 8.298
    },
    "yield" : {
           "Zusammensetzung" : 86, # multiply by dry matter perc
            "Emissionsfaktor": 34.6
    },
    "FUEL": {
        "Zusammensetzung" : 1.0,
        "Emissionsfaktor": 3.122
    }
}

In [14]:
# Cell 1: Import Libraries
import geopandas as gpd
import pandas as pd
import glob
import os
from shapely.geometry import MultiPoint

# Set SHAPE_RESTORE_SHX environment variable
os.environ["SHAPE_RESTORE_SHX"] = "YES"
gemmingen_area = 19.18 #per Ha

# Cell 2: Define Root Folder and Initialize List
root_folder = "data/Erntejahr 2022 (Weizen, Düngung)"

# Initialize an empty list to store shapefile paths
shapefile_list = []

# Traverse the directory structure
for subdir, _, _ in os.walk(root_folder):
    # Look for .shp files in the 'doc' folder of each subdirectory
    shapefiles = glob.glob(os.path.join(subdir, "doc", "*.shp"))
    shapefile_list.extend(shapefiles)


# Cell 3: Read Shapefiles and Combine into a Single GeoDataFrame
gdfs = []
for shapefile_path in shapefile_list:
    gdf = gpd.read_file(shapefile_path)
    # Append the GeoDataFrame to a list
    gdfs.append(gdf)

# Concatenate all GeoDataFrames into a single GeoDataFrame
combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)

# Fill missing values with NA
combined_gdf = combined_gdf.fillna(0)

# Cell 4: Display Output and Columns
print("All shapefiles combined successfully!")
# Uncomment the lines below if you want to save the combined shapefile
# Cell 5: Save Combined GeoDataFrame as a Shapefile
# output_shapefile_path = os.path.join(root_folder, "combined_shapefile.shp")
# combined_gdf.to_file(output_shapefile_path)


All shapefiles combined successfully!


In [15]:
def getValues(column):
    percent = calculate_emissions_json[column]["Zusammensetzung"]
    emissionfactor = calculate_emissions_json[column]["Emissionsfaktor"]
    return percent, emissionfactor

In [16]:
# combined_gdf['Product'] = combined_gdf['Product'].apply(lambda x: x.split()[0])

# Cell 6: Calculate Emissions for Each Product
def calculate_emissions(row):
    product = row["Product"]
    appliedRate = row["AppliedRate"]
    if product in calculate_emissions_json:
        if calculate_emissions_json[product]["Zusammensetzung"]:
            if isinstance(calculate_emissions_json[product]["Zusammensetzung"], dict):
                composition = calculate_emissions_json[product]["Zusammensetzung"]["N"]
            else:
                composition = calculate_emissions_json[product]["Zusammensetzung"]
            emissions_factor = calculate_emissions_json[product]["Emissionsfaktor"]
            emissions = composition * appliedRate * emissions_factor
        else:
            emissions = appliedRate * calculate_emissions_json[product]["Emissionsfaktor"]
    else:
        emissions = 0
    return emissions

combined_gdf["Emissions"] = combined_gdf.apply(calculate_emissions, axis=1)
combined_gdf["FuelEmissions"] = combined_gdf["FUEL"] * calculate_emissions_json["FUEL"]["Emissionsfaktor"]
if "VRYIELDMAS" in combined_gdf.columns and "DRYMATTER" in combined_gdf.columns:
    combined_gdf["YieldEmissions"] = combined_gdf["VRYIELDMAS"] * combined_gdf["DRYMATTER"] * calculate_emissions_json["yield"]["Zusammensetzung"] * calculate_emissions_json["yield"]["Emissionsfaktor"]
else:
    combined_gdf["YieldEmissions"] = 0
# combined_gdf["YieldEmissions"] = combined_gdf["VRYIELDMAS"] *  combined_gdf["DRYMATTER"] * calculate_emissions_json["yield"]["Zusammensetzung"] * calculate_emissions_json["yield"]["Emissionsfaktor"]

In [17]:
average_emissions = combined_gdf['Emissions'].mean() + combined_gdf['FuelEmissions'].mean() + combined_gdf['YieldEmissions'].mean()
print(average_emissions * gemmingen_area)

4692.094500322628


In [18]:
combined_gdf_grouped = combined_gdf.groupby("Product").agg({"Emissions": "sum"}) * gemmingen_area
combined_gdf_grouped

,Emissions
Product,
ASS 26N,7.638323e+07
Gärsubstrat,0.000000e+00
Kalkamonsalpeter 27,4.072505e+08
